In [ ]:
import numpy as np
import astropy
from astropy import cosmology
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
import scipy

In [ ]:
r_mag = [16.08, 16.43, 16.15, 16.67, 17.98, 17.42, 16.41, 16.57, 17.22, 18.47]
redshift = [0.0868, 0.0865, 0.0901, 0.0897, 0.0993, 0.0994, 0.1122, 0.1120, 0.0887, 0.0889]
dist_mod = []
f_cgs = []
bol_lum = []
agn_mass = []
vL_v = []
L_nu_list = []
scaled_mass = []


lambda_r_A = 6261.0  #angstroms       
lambda_r_cm = lambda_r_A * 1e-8 #cm
c_cms = 2.99792458e10 #cm/s not m/s
nu_obs = c_cms / lambda_r_cm #i was getting weird values and asked copilot on what i should do and chat and they said to add this, what is this?

BC = 10.0 #in the sdss catalog
nMgy_to_Jy = 3.631e-6   #adding this conversion

cm = FlatLambdaCDM(H0=70, Om0=0.3) #for the distance modulus
for val in redshift:
    d_l = cm.luminosity_distance(val)
    d_L_cm = d_l.to(u.centimeter).value  #was getting error so I need to change the Mpc to cm
    dist_mod.append(d_L_cm)

print(*dist_mod, sep = '\n')

ModuleNotFoundError: No module named 'scipy.special'

Now converting the mag that we were given into Jansky's so that we can get the flux from it

In [ ]:
for val in r_mag:
    f_nmgy = 10**((22.5-val)/2.5)
    f_jnsky = f_nmgy * nMgy_to_Jy
    f_v = f_jnsky * 10**(-23)
    f_cgs.append(f_v) #in erg s^-1 Hz^-1

print(*f_cgs, sep='\n')

Now, we are gonna get the bolometric luminosity

In [ ]:
#L = 4pir^2(f_cgs)

for val,num, z in zip(f_cgs, dist_mod, redshift):
    L_nu = 4.0 * np.pi * (num**2) * val * (1.0 + z)
    L_nu_list.append(L_nu)
for z, val in zip(redshift, L_nu_list):
    nu_rest = nu_obs * (1.0 + z)
    nuLnu = nu_rest * val
    vL_v.append(nuLnu)
for val in vL_v:
    Lbol = BC * val
    bol_lum.append(Lbol)

print(*bol_lum, sep='\n')

print("Now the v_l_v") #when we add the effective wavelength into the calculation becuase of the the power of the band
print(*vL_v, sep='\n')


Now using the Eddington luminosity limit and the eddington ratio for AGNs

In [ ]:
edd_ratio = 0.1 #using this value for now and then moving forward

for val in bol_lum:
    m_bh = val / (edd_ratio * 1.26 * 10**(38))
    agn_mass.append(m_bh)


for num in agn_mass:
    scaled = num / 1e8
    print(f"{scaled:.2f}e8 * M_sun")
    

Now, we should focus on converting the h*kpc into pc for our simulations

Now we are looking at getting FITS files for our AGN - the SDSS AGN pairs has a designation which has RA in hour angle and Dec in degrees and I need to convert both into degrees and write it in an ASCii file to put into WISE

In [ ]:
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.table import Table
from astropy.io import ascii

All the data

In [ ]:
agn_loc_hourangle = ["00:02:49.07 + 00:45:04.8", "00:02:49.44 + 00:45:06.7", "00:02:57.21 + 00:07:50.5 ", "00:02:58.59 + 00:08:31.0", 
           "00:03:23.74 + 01:05:47.3", "00:03:23.74 + 01:05:59.5", "00:04:25.78 - 09:58:54.4", "00:04:26.66 - 09:58:34.4", "00:04:31.92 - 01:14:11.7", "00:04:33.25 - 01:13:34.4"]
agn_final_loc = []

In [ ]:
ra_col_data = []
dec_col_data = []

for val in agn_loc_hourangle:
    transformation = SkyCoord(val.strip(), unit = (u.hourangle, u.deg), frame = 'icrs') #have to make both into degrees
    agn_final_loc.append([transformation.ra.deg, transformation.dec.deg])
    ra_col_data.append(transformation.ra.deg)
    dec_col_data.append(transformation.dec.deg)
for i in range(len(ra_col_data)):
    agn_final_loc.append([ra_col_data[i], dec_col_data[i]])


print(*agn_final_loc, sep='\n')

In [ ]:

ipac_cat = Table([ra_col_data, dec_col_data], names=('ra', 'dec'))
ascii.write(ipac_cat, 'for_wise_search3.dat', format='ipac', overwrite=True)



Now I have the FITS file - i will not retrieve my code from 1 year and a half ago to see if I can visualize this as needed

In [ ]:
import matplotlib
import matplotlib.text

matplotlib.text.Text(text=  'x', fontproperties= 'Times New Roman')  # lol, still in process in figuriing out on how to change the font

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits
from astropy.wcs import WCS
from astropy.visualization import astropy_mpl_style
from astropy.visualization.mpl_normalize import simple_norm

#class i made in spring 24

class FITS_analyzer:
    def __init__(self, file):
        self.file = file
    def analyze(self):
        with fits.open(self.file) as hdul:
            image_data = hdul[0].data
            wcs = WCS(hdul[0].header)


        ysize, xsize = image_data.shape

        xcorner, ycorner = wcs.pixel_to_world_values([0, xsize], [0, ysize])


        fig, ax = plt.subplots()


        extent = [xcorner[0], xcorner[1], ycorner[0], ycorner[1]]
        norm = simple_norm(image_data, stretch='log', power=2.0, asinh_a=0.5)
        im = ax.imshow(image_data, cmap='magma', norm=norm, origin='upper', aspect='equal', extent=extent)


        x_ticks = np.linspace(xcorner[0], xcorner[1], 8)
        y_ticks = np.linspace(ycorner[0], ycorner[1], 8)

        ax.set_xticks(x_ticks)
        ax.set_yticks(y_ticks)

        ax.set_xticklabels([f'{val:.2f}' for val in x_ticks], fontsize=10)  
        ax.set_yticklabels([f'{val:.2f}' for val in y_ticks], fontsize=10)  
        ax.tick_params(axis='both', direction='in', length=6, width=1, which='both', top=True, right=True)


        ax.set_xlabel('Right Ascension (degrees)', fontsize=12)  # Specify font size for x-axis label
        ax.set_ylabel('Declination (degrees)', fontsize=12)  # Specify font size for y-axis label
        ax.grid(False)



        cbar = plt.colorbar(im, ax=ax, label='Intensity')
        cbar.ax.tick_params(labelsize=8)  # Specify smaller font size for colorbar ticks
        cbar.set_label('Intensity', fontsize=12)  # Specify font size for colorbar label


file_path = 'image_2MASS-K_s.fits' #had to use relative path, need to update class so that any input(from downloads) will be able to output
analyzer = FITS_analyzer(file_path)
analyzer.analyze()
    
    
            




Now building the pipeline to input straight from paper to convert to ASCii

/data/ folder has the pipeline to clean data and choose columns that are needed - so we can directly import the cleaned data to make it in a ASCii

In [ ]:
import pandas as pd
import os

csv_file = os.path.join("data", "data/Dataframe_csv", "data_sdss_agn_minimized.csv")

df = pd.read_csv(csv_file)
df.head()

In [ ]:
print(df.columns)

In [ ]:
ra_col_df = []
dec_col_df = []

for val in df['SDSS designation']:
    val_clean = val.lstrip("J")  


    ra_raw = val_clean[:9]   
    dec_raw = val_clean[9:]  


    ra_h = ra_raw[:2]
    ra_m = ra_raw[2:4]
    ra_s = ra_raw[4:]

    dec_sign = '+' if dec_raw[0] != '-' else '-'
    dec_d = dec_raw[1:3]
    dec_m = dec_raw[3:5]
    dec_s = dec_raw[5:]

    ra_formatted = f"{ra_h}:{ra_m}:{ra_s}"
    dec_formatted = f"{dec_sign}{dec_d}:{dec_m}:{dec_s}"


    coord = SkyCoord(ra=ra_formatted, dec=dec_formatted, unit = (u.hourangle, u.deg), frame='icrs')

    ra_col_df.append(coord.ra.deg)
    dec_col_df.append(coord.dec.deg)


dataframe_final = pd.DataFrame({
    'ra': ra_col_df,
    'dec': dec_col_df
})

print(dataframe_final)

In [ ]:
# ipac_cat = Table.from_pandas(dataframe_final[['ra', 'dec']])
# ascii.write(ipac_cat, 'for_wise_search1_allquant.dat', format='ipac', overwrite=True)

